In [1]:
from gurobipy import *
import numpy as np
import pandas as pd
from scipy.spatial import distance
from itertools import chain, combinations
import matplotlib.pyplot as plt
import os, sys
import readData as dt


In [2]:
dt = dt.Data()
dt.get_data("input/P/P-n19-k2.vrp")

dt.X

[30, 37, 49, 52, 31, 52, 42, 52, 57, 62, 42, 27, 43, 58, 37, 61, 62, 63, 45]

In [3]:
#Set
C = [] #set of customers
C_0 = [] #set of customers and depot
C_plus = [] #set of customers and copy of depot
N = [] #set of all nodes
K = [] #set of trucks
SD = [] #set of small drones
CD = [] #set of cargo drones
D = [] #set of customers' demand
CL = [] #set of low-perishability products customers
CH = [] #set of high-perishability products customers
#

T = None #completion time
QT = None #truck capacity
QCD = None #cargo-drone capacity
QSD = None #small drone capacity


#



In [4]:
dt.num_of_truck = 1
dt.num_of_cargo_drone = 1
dt.num_of_small_drone = 0

In [5]:
n = dt.num_of_customer - 1

C = [i for i in range(1, n+1)]
C_0 = [i for i in range(0, n+1)]
C_plus = [i for i in range(1, n + 2)]
N = [i for i in range(0, n+2)]
K = [i for i in range(dt.num_of_truck)]
CD = [i for i in range(dt.num_of_cargo_drone)]
SD = [i for i in range(dt.num_of_small_drone)]
D = dt.demand_matrix
D.append(0)
CL = [i for i in range(1,10)]
CH = [i for i in range(10,19)]
QT = dt.truck_capacity
QCD = dt.cargo_drone_capacity
QSD = dt.small_drone_capacity
TL = 10000
TH = 10000

cost_t = dt.distance_matrix
cost_td = dt.drone_time_matrix
M = GRB.MAXINT


AttributeError: 'Data' object has no attribute 'cargo_drone_capacity'

In [ ]:
#variables
x = {}
xsd = {}
xcd = {}
y = {}
ysd = {}
ycd = {}
t = {}
tsd = {}
tcd = {}
u = {}
ucd = {}

model = Model('VRPTD')

In [ ]:

#DECISION VARIABLES
for k in K:
    for i in N:
        y[k,i] = model.addVar(0, 1, vtype = GRB.BINARY,name = "y%d,%d" %(k,i))
        for j in N:
            x[k,i,j] = model.addVar(0, 1, vtype = GRB.BINARY,name = "x%d,%d,%d" %(k,i,j))

for cd in CD:
    for i in N:
        ycd[cd,i] = model.addVar(0, 1, vtype = GRB.BINARY,name = "ycd%d,%d" %(cd,i))
        for j in N:
            xcd[cd,i,j] = model.addVar(0, 1, vtype = GRB.BINARY,name = "xcd%d,%d,%d" %(cd,i,j))
            
for sd in SD:
    for i in C:
        ysd[sd,i] = model.addVar(0, 1, vtype = GRB.BINARY,name = "ysd%d,%d" %(sd,i))
#         xsd[sd,i] = model.addVar(0, 1, vtype = GRB.BINARY,name = "xsd%d,%d" %(sd,i))

for k in K:
    for i in N:
        t[k,i] = model.addVar(0, GRB.INFINITY, 1.0, GRB.CONTINUOUS, name = "t%d,%d" %(k,i))
        
for cd in CD:
    for i in N:
        tcd[cd,i] = model.addVar(0, GRB.INFINITY, 1.0, GRB.CONTINUOUS, name = "tcd%d,%d" %(cd,i))
        
for sd in SD:
    for i in N:
        tsd[sd,i] = model.addVar(0, GRB.INFINITY, 1.0, GRB.CONTINUOUS, name = "tsd%d,%d" %(sd,i))
for k in K:
    for i in N:
        u[k,i] = model.addVar(0, GRB.INFINITY, vtype = GRB.CONTINUOUS, name = "u%d" %i)
for cd in CD:
    for i in N:
        ucd[cd, i] = model.addVar(0, GRB.INFINITY, vtype = GRB.CONTINUOUS, name = "ucd%d" %i)
        

        
alpha = model.addVar(0, GRB.INFINITY, 1.0, GRB.CONTINUOUS, "traveltime")
expr = LinExpr(0)
expr.addTerms(1.0, alpha)

model.setObjective(expr, GRB.MINIMIZE)
#objective
tmp_expr0 = LinExpr(0)
tmp_expr1 = LinExpr(0)
tmp_expr2 = LinExpr(0)
for k in K:
    tmp_expr0.addTerms(1.0, t[k, n+1])
for cd in CD:
    tmp_expr1.addTerms(1.0, tcd[cd, n+1])
for sd in SD:
    for i in C:
        tmp_expr2.addTerms(1.0, tsd[sd, i])
model.addConstr(alpha >= tmp_expr0 + tmp_expr1 + tmp_expr2)
# model.update()
# print(tmp_expr0)
# print(tmp_expr1)
# print(tmp_expr2)
tmp_expr0 = LinExpr(0)
tmp_expr1 = LinExpr(0)
tmp_expr2 = LinExpr(0)
for k in K:
     for i in C_0:
            for j in C_plus:
                tmp_expr0.addTerms(cost_t[i][j], x[k, i,j])
for cd in CD:
     for i in C_0:
            for j in C_plus:
                    tmp_expr1.addTerms(cost_td[i][j], xcd[cd, i,j])

for sd in SD:
    for i in C:
        tmp_expr2.addTerms(cost_td[0][i], ysd[sd, i])
model.addConstr(alpha >= tmp_expr0 + tmp_expr1 + tmp_expr2)

#2
for i in C:
    tmp_expr0 = LinExpr(0)
    tmp_expr1 = LinExpr(0)
    tmp_expr2 = LinExpr(0)
    for k in K:
        tmp_expr0.addTerms(1.0, y[k,i])
    for cd in CD:
        tmp_expr1.addTerms(1.0, ycd[cd,i])
    for sd in SD:
        tmp_expr2.addTerms(1.0, ysd[sd,i])
#     model.update()
#     print(tmp_expr0)
    model.addConstr(tmp_expr0 + tmp_expr1 + tmp_expr2 == 1)
#3
for k in K:
    tmp_expr = LinExpr(0)
    for i in C_plus:
        tmp_expr.addTerms(1.0, x[k,0,i])
    model.addConstr(tmp_expr == 1)
    tmp_expr.clear()

#4
for k in K:
    tmp_expr = LinExpr(0)
    for i in C_0:
        tmp_expr.addTerms(1.0, x[k,i,n+1])
    model.addConstr(tmp_expr == 1)
    tmp_expr.clear()
    
#5
for i in C:
    for k in K:
        tmp_expr0 = LinExpr(0)
        tmp_expr1 = LinExpr(0)
        for j in C_0:
            tmp_expr0.addTerms(1.0, x[k,j,i])
        for j in C_plus:
            tmp_expr1.addTerms(1.0, x[k,i,j])
        model.addConstr(tmp_expr1 == tmp_expr0) 
        model.addConstr(y[k,i] == tmp_expr0) 
        model.addConstr(y[k,i] == tmp_expr1) 
        tmp_expr0.clear()
        tmp_expr1.clear()

#6
for cd in CD:
    tmp_expr = LinExpr(0)
    for i in C_plus:
        tmp_expr.addTerms(1.0, xcd[cd,0,i])
    model.addConstr(tmp_expr == 1)
    tmp_expr.clear()

#7
for cd in CD:
    tmp_expr = LinExpr(0)
    for i in C_0:
        tmp_expr.addTerms(1.0, xcd[cd,i,n+1])
    model.addConstr(tmp_expr == 1)
    tmp_expr.clear()
    
#8
for i in C:
    for cd in CD:
        tmp_expr0 = LinExpr(0)
        tmp_expr1 = LinExpr(0)
        for j in C_0:
            tmp_expr0.addTerms(1.0, xcd[cd,j,i])
        for j in C_plus:
            tmp_expr1.addTerms(1.0, xcd[cd,i,j])
        model.addConstr(tmp_expr1 == tmp_expr0) 
        model.addConstr(ycd[cd,i] == tmp_expr0)
        model.addConstr(ycd[cd,i] == tmp_expr1)
        tmp_expr0.clear()
        tmp_expr1.clear()
#9
for sd in SD:
    tmp_expr = LinExpr(0)
    for i in C:
        tmp_expr.addTerms(1.0, ysd[sd,i])
    model.addConstr(tmp_expr <= 1)
    
#10

for k in K:
    tmp_expr = LinExpr(0)
    for i in C:
        tmp_expr.addTerms(D[i], y[k,i])
    model.addConstr(tmp_expr <= QT)
    tmp_expr.clear()
    
#11
for cd in CD:
    tmp_expr = LinExpr(0)
    for i in C:
        tmp_expr.addTerms(D[i], ycd[cd,i])
    model.addConstr(tmp_expr <= QCD)
    tmp_expr.clear()
        
#12
for sd in SD:
    tmp_expr = LinExpr(0)
    for i in C:
        tmp_expr.addTerms(D[i], ysd[sd,i])
    model.addConstr(tmp_expr <= QSD)
    tmp_expr.clear()
#13 arrival time

for i in C_0:
    for j in C_plus:
        for k in K:
            model.addConstr(t[k,j] >= t[k,i] + cost_t[i][j] - M*(1-x[k,i,j]))
#14
for i in C_0:
    for j in C_plus:
        for cd in CD:
            model.addConstr(tcd[cd,j] >= tcd[cd,i] + cost_td[i][j] - M*(1-xcd[cd,i,j]))

#15
for i in C:
    for sd in SD:
        model.addConstr(tsd[sd,i] >= cost_td[0][i] + cost_td[i][n+1]  - M*(1 - ysd[sd,i]) )
                            
# for i in C:
#         for sd in SD:
#             model.addConstr(tsd[sd,i] == ysd[sd,i]*cost_td[0][i])           
# 16

for i in CL:
    tmp_expr0 = QuadExpr(0)
    tmp_expr1 = QuadExpr(0)
    tmp_expr2 = QuadExpr(0) 
   
    for k in K:
        tmp_expr0.addTerms(1.0, t[k,i], y[k,i])
      
    for cd in CD:
        tmp_expr1.addTerms(1.0, tcd[cd,i], ycd[cd,i])
        
    for sd in SD:
        tmp_expr2.addTerms(1.0, tsd[sd,i], ysd[sd,i])
        
        
    model.addConstr(tmp_expr0 + tmp_expr1 + tmp_expr2 <= TL)
                    
#17
for i in CH:
    tmp_expr0 = QuadExpr(0)
    tmp_expr1 = QuadExpr(0)
    tmp_expr2 = QuadExpr(0)
    for k in K:
        tmp_expr0.addTerms(1.0, t[k,i], y[k,i])
        model.update()
        print (tmp_expr0)
    for cd in CD:
        tmp_expr1.addTerms(1.0, tcd[cd,i], ycd[cd,i])
        
    for sd in SD:
        tmp_expr2.addTerms(1.0, tsd[sd,i], ysd[sd,i])
        
    model.addConstr(tmp_expr0 + tmp_expr1 + tmp_expr2 <= TH)

#18
for i in N:
    for j in N:
        for k in K:
            tmp0 = u[k,i] - u[k,j] + QT*x[k,i,j]
            tmp1 = QT - D[j]
            
            model.addConstr(tmp0 <= tmp1)
#19

for i in N:
    for k in K:
        model.addConstr( u[k,i] >= D[i])
        model.addConstr( u[k,i] <= QT)

#20
for i in N:
    for j in N:
        for cd in CD:
            tmp0 = ucd[cd,i] - ucd[cd,j] + QCD*xcd[cd,i,j]
            tmp1 = QCD - D[j]
            
            model.addConstr(tmp0 <= tmp1)                     
                     
#21
for i in N:
    for cd in CD:
        model.addConstr( ucd[cd,i] >= D[i])
        model.addConstr( ucd[cd,i] <= QCD)
                    
#domain constraints
# for k in K:
#     for j in C:
#         model.addConstr(t[k,j] >= 0)
# for cd in CD:
#     for j in C:
#         model.addConstr(tcd[cd,j] >= 0)
# for sd in SD:
#     for j in C:
#         model.addConstr(tsd[sd,j] >= 0)

In [ ]:

model.Params.MIPFocus = 1
model.optimize()

In [ ]:
model.printAttr('x')

In [ ]:
dt.X[0]

In [ ]:

truck_tours = [[] for k in K]


for k in K:
    truck_tours[k] = [(i,j) for i in C_0 for j in C_plus if x[k,i,j].x > 0.99 ]
plt.plot(dt.X[0], dt.Y[0], c='r', marker='s')

for i in range(1, dt.num_of_customer):
    plt.scatter(dt.X[i], dt.Y[i], c='b')
    plt.annotate(str("  ") + str(int(i)), (dt.X[i], dt.Y[i]))
    
for k in K:
    for i, j in truck_tours[k]:
        if j != dt.num_of_customer:
            plt.plot([dt.X[i], dt.X[j]], [dt.Y[i], dt.Y[j]], c='g', zorder=0)
        else:
            for k, h in truck_tours[k]:
                if h == i:
                    plt.plot([dt.X[i], dt.X[0]], [dt.Y[i], dt.Y[0]], c='g', zorder=0)

cargo_drone_tours = [[] for cd in CD]


for cd in CD:
    cargo_drone_tours[cd] = [(i,j) for i in C_0 for j in C_plus if xcd[cd,i,j].x > 0.99 ]

for cd in CD:
    for i, j in cargo_drone_tours[cd]:
        if j != dt.num_of_customer:
            plt.plot([dt.X[i], dt.X[j]], [dt.Y[i], dt.Y[j]], c='r', zorder=0)
        else:
            for k, h in cargo_drone_tours[cd]:
                if h == i:
                    plt.plot([dt.X[i], dt.X[0]], [dt.Y[i], dt.Y[0]], c='r', zorder=0)
                    
for sd in SD:
    default_width= 1
    for i in C:
        if ysd[sd,i].x > 0.99:
            plt.plot([dt.X[0],dt.X[i]], [dt.Y[0], dt.Y[i]], color='black', linestyle='dashed', linewidth = default_width)
    

cargo_drone_tours = [[] for cd in CD]

